# Sudoku

Writing an agent to solve every diagonal sudoku puzzle. A diagonal sudoku is like a regular sudoku, except that among the two main diagonals, the numbers 1 to 9 should all appear exactly once. 

### 1. Encoding the board
#### Rows and Columns

* The rows will be labelled by the letters A, B, C, D, E, F, G, H, I.
* The columns will be labelled by the numbers 1, 2, 3, 4, 5, 6, 7, 8, 9. 
* The 3x3 squares won't be labelled.

#### Boxes, Units and Peers

* The individual squares at the intersection of rows and columns will be called boxes. These boxes will have labels 'A1', 'A2', ..., 'I9'.
* The complete rows, columns, 3x3 squares, and the two main diagonals will be called units. Thus, each unit is a set of 9 boxes, and there are 29 units in total.
* For a particular box (such as 'A1'), its peers will be all other boxes that belong to a common unit (namely, those that belong to the same row, column, 3x3 square or the main diagonal).

In [1]:
import numpy as np

# Encoding the board
rows = 'ABCDEFGHI'
cols = '123456789'

# Cross product of elements in A and elements in B
def cross(A, B):
    return [s+t for s in A for t in B]

boxes = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
diagonal_units = [['A1', 'B2', 'C3', 'D4', 'E5', 'F6', 'G7', 'H8', 'I9'], 
                  ['A9', 'B8', 'C7', 'D6', 'E5', 'F4', 'G3', 'H2', 'I1']]
unitlist = row_units + column_units + square_units + diagonal_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [2]:
# Make sure diagonal units work
print (len(peers['A1']))
print (len(peers['E5']))
print (len(peers['A2']))

26
32
20


In [3]:
# Convert grid into a dict of {square: char} with '123456789' for empties
def grid_values(grid):
    chars = []
    digits = '123456789'
    for c in grid:
        if c in digits:
            chars.append(c)
        if c == '.':
            chars.append(digits)
    assert len(chars) == 81
    return dict(zip(boxes, chars))

In [4]:
assignments = []

# Assigns a value to a given box. If it updates the board record it
def assign_value(values, box, value):
    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values

In [5]:
# Display the values as a 2-D grid
def display(values):
    width = 1 + max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r + c].center(width) + ('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    print()

In [6]:
# Make sure display works
grid1 = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
display(grid_values(grid1))

    2     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789     6     |    2     123456789 123456789 
123456789 123456789     1     |123456789 123456789 123456789 |123456789     7     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     6     |123456789 123456789     8     |123456789 123456789 123456789 
    3     123456789 123456789 |123456789     9     123456789 |123456789 123456789     7     
123456789 123456789 123456789 |    6     123456789 123456789 |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     4     123456789 |123456789 123456789 123456789 |    8     123456789 123456789 
123456789 123456789     5     |    2     123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |12345678

### 2. Implementing Strategies

#### Strategy 1: Elimination
If a box has a value assigned, then none of the peers of this box can have this value.

In [7]:
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            assign_value(values, peer, values[peer].replace(digit, ''))
    return values

In [8]:
display(eliminate(grid_values(grid1)))

   2     356789  34789  |1345789  134578  134579 | 13569  1345689   145   
 45789     57    34789  |1345789  134578    6    |   2      1345   14589  
 45689   35689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2     134678  13479  |  1679     46     1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    



#### Strategy 2: Naked Twins
If there are two unsolved boxes in a unit and there are two digits that can only go in the same two boxes, then no others peers in that unit can have the two digits. 

In [9]:
from collections import Counter

def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    """
    for unit in unitlist:
        two_values = [values[box] for box in unit if len(values[box]) == 2]
        twins = [twin for twin, n in Counter(two_values).items() if n == 2]
        if len(twins) > 0:
            for twin in twins:
                for box in unit:
                    if values[box] != twin:
                        for digit in twin:
                            if digit in values[box]:
                                assign_value(values, box, values[box].replace(digit, ''))
    return values

In [10]:
# Make sure naked_twins works - Test 1
values = {'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8',
          'H5': '6', 'F9': '7', 'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8',
          'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23', 'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5',
          'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'A4': '2357', 'A7': '27',
          'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6',
          'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2',
          'F6': '125', 'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '379', 'F1': '6',
          'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'E2': '37', 'F7': '35', 'F8': '9',
          'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17', 'D3': '2379', 'B4': '27',
          'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'D6': '279',
        'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}
print("Before:")
display(values)
print("After:")
display(naked_twins(values))

Before:
  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1   2379| 237  347  279 |  34   8    6  
  8    37  379 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17   6    17 |  8    2    3  
  23   6    23 |  8    5    4  |  1    7    9  

After:
  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1    79 | 237  347  279 |  34   8    6  
  8    3    79 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17   6

In [11]:
# Make sure naked_twins works - Test 2
values = {'A1': '23', 'A2': '4', 'A3': '7', 'A4': '6', 'A5': '8', 'A6': '5', 'A7': '23', 'A8': '9',
          'A9': '1', 'B1': '6', 'B2': '9', 'B3': '8', 'B4': '4', 'B5': '37', 'B6': '1', 'B7': '237',
          'B8': '5', 'B9': '237', 'C1': '23', 'C2': '5', 'C3': '1', 'C4': '23', 'C5': '379',
          'C6': '2379', 'C7': '8', 'C8': '6', 'C9': '4', 'D1': '8', 'D2': '17', 'D3': '9',
          'D4': '1235', 'D5': '6', 'D6': '237', 'D7': '4', 'D8': '27', 'D9': '2357', 'E1': '5',
          'E2': '6', 'E3': '2', 'E4': '8', 'E5': '347', 'E6': '347', 'E7': '37', 'E8': '1', 'E9': '9',
          'F1': '4', 'F2': '17', 'F3': '3', 'F4': '125', 'F5': '579', 'F6': '279', 'F7': '6',
          'F8': '8', 'F9': '257', 'G1': '1', 'G2': '8', 'G3': '6', 'G4': '35', 'G5': '345',
          'G6': '34', 'G7': '9', 'G8': '27', 'G9': '27', 'H1': '7', 'H2': '2', 'H3': '4', 'H4': '9',
          'H5': '1', 'H6': '8', 'H7': '5', 'H8': '3', 'H9': '6', 'I1': '9', 'I2': '3', 'I3': '5',
          'I4': '7', 'I5': '2', 'I6': '6', 'I7': '1', 'I8': '4', 'I9': '8'}
print("Before:")
display(values)
print("After:")
display(naked_twins(values))

Before:
  23   4    7  |  6    8    5  |  23   9    1  
  6    9    8  |  4    37   1  | 237   5   237 
  23   5    1  |  23  379  2379|  8    6    4  
---------------+---------------+---------------
  8    17   9  | 1235  6   237 |  4    27  2357
  5    6    2  |  8   347  347 |  37   1    9  
  4    17   3  | 125  579  279 |  6    8   257 
---------------+---------------+---------------
  1    8    6  |  35  345   34 |  9    27   27 
  7    2    4  |  9    1    8  |  5    3    6  
  9    3    5  |  7    2    6  |  1    4    8  

After:
  23   4    7  |  6    8    5  |  23   9    1  
  6    9    8  |  4    3    1  | 237   5   237 
  23   5    1  |  23   79   79 |  8    6    4  
---------------+---------------+---------------
  8    17   9  | 1235  6   237 |  4    27  2357
  5    6    2  |  8   347  347 |  37   1    9  
  4    17   3  | 125  579  279 |  6    8   257 
---------------+---------------+---------------
  1    8    6  |  35  345   34 |  9    27   27 
  7    2    4  |  9    1

#### Strategy 3: Only Choice
If there is only one box in a unit which would allow a certain digit, then that box must be assigned that digit.

In [12]:
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                assign_value(values, dplaces[0], digit)
    return values

In [13]:
display(only_choice(naked_twins(eliminate(grid_values(grid1)))))

   2     356789  34789  |1345789  134578  134579 | 13569  1345689   145   
 45789     57    34789  |1345789  134578    6    |   2      1345   14589  
 45689   35689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |   4     123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4       2    | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2     134678  13479  |  1679     6      1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    



#### Applying Constraint Propagation

In [16]:
def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    """
    stalled = False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = naked_twins(values)
        values = only_choice(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        stalled = solved_values_before == solved_values_after
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [17]:
# A simple sudoku
print('Grid1:')
grid1 = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
display(reduce_puzzle(grid_values(grid1)))

# A harder sudoku
print('Grid2:')
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
display(reduce_puzzle(grid_values(grid2)))

Grid1:
2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 

Grid2:


TypeError: 'bool' object is not subscriptable

#### Search
Pick a box with a minimal number of possible values. Try to solve each of the puzzles obtained by choosing each of these values, recursively.

In [18]:
# Using depth-first search and propagation, try all possible values
def search(values):
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [19]:
# A simple sudoku
print('Grid1:')
grid1 = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
display(search(grid_values(grid1)))

# A harder sudoku
print('Grid2:')
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
display(search(grid_values(grid2)))

Grid1:
2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 

Grid2:


TypeError: 'bool' object is not subscriptable

### 3. Testing Sudokus

In [20]:
def solve(grid):
    """
    Find the solution to a Sudoku grid.
    Args:
        grid(string): a string representing a sudoku grid.
            Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns:
        The dictionary representation of the final sudoku grid. False if no solution exists.
    """
    return search(grid_values(grid))

if __name__ == '__main__':
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    display(solve(diag_sudoku_grid))

    try:
        from visualize import visualize_assignments
        visualize_assignments(assignments)

    except SystemExit:
        pass
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 

We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.
